## Experement Presentation by Team_A

## Objectives

- surmarize a normal passage a combination of extractive and abstractive techniques then after apply it on googles T5 text summarizer

- adjust text leanght and observe the outcomes

- apply the above  on a data frame

- demonstrate a question and answer Model

- Apply the question and answer model on a data frame

### 1  Setting up necessary libraries

In [141]:
import numpy as np
from scipy import stats
import pandas as pd

#models
from allennlp.predictors.predictor import Predictor
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from summarizer import Summarizer
predictor = Predictor.from_path("C:/Users/Silas_Dell/Downloads/Compressed/bidaf-elmo.2021-02-11.tar_2.gz")

# Suppressing unnwarranted warnings
import warnings
warnings.filterwarnings("ignore")

error loading _jsonnet (this is expected on Windows), treating C:\Users\SILAS_~1\AppData\Local\Temp\tmppjg4of76\config.json as plain json


In [142]:
# Load Model and Tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
tokenizer = AutoTokenizer.from_pretrained('t5-base')

### Input Text



In [143]:
text = """
New York (CNN Business)Netflix is synonymous with streaming, but its competitors have a distinct advantage that threatens the streaming leader's position at the top.
Disney has Disney+, but it also has theme parks, plush Baby Yoda dolls, blockbuster Marvel movies and ESPN. Comcast (CMCSA), Amazon (AMZN), ViacomCBS (VIACA), CNN's parent company WarnerMedia and Apple (AAPL) all have their own streaming services, too, but they also have other forms of revenue.
As for Netflix (NFLX), its revenue driver is based entirely on building its subscriber base. It's worked out well for the company - so far. But it's starting to look like the king of streaming will soon need something other than new subscribers to keep growing.
The streaming service reported Tuesday it now has 208 million subscribers globally, after adding 4 million subscribers in the first quarter of 2021. But that number missed expectations and the forecasts for its next quarter were also pretty weak.
That was a big whiff for Netflix - a company coming off a massive year of growth thanks in large part to the pandemic driving people indoors - and Wall Street's reaction has not been great.
The company's stock dropped as much as 8% on Wednesday, leading some to wonder what the future of the streamer looks like if competition continues to gain strength, people start heading outdoors and if, most importantly, its growth slows.
"If you hit a wall with [subscriptions] then you pretty much don't have a super growth strategy anymore in your most developed markets," Michael Nathanson, a media analyst and founding partner at MoffettNathanson, told CNN Business. "What can they do to take even more revenue out of the market, above and beyond streaming revenues?"
Or put another way, the company's lackluster user growth last quarter is a signal that it wouldn't hurt if Netflix - a company that's lived and died with its subscriber numbers - started thinking about other ways to make money.
An ad-supported Netflix? Not so fast
There are ways for Netflix to make money other than raising prices or adding subscribers. The most obvious: selling advertising.
Netflix could have 30-second commercials on their programming or get sponsors for their biggest series and films. TV has worked that way forever, why not Netflix?
That's probably not going to happen, given that CEO Reed Hastings has been vocal about the unlikelihood of an ad-supported Netflix service. His reasoning: It doesn't make business sense.
"It's a judgment call... It's a belief we can build a better business, a more valuable business [without advertising]," Hastings told Variety in September. "You know, advertising looks easy until you get in it. Then you realize you have to rip that revenue away from other places because the total ad market isn't growing, and in fact right now it's shrinking. It's hand-to-hand combat to get people to spend less on, you know, ABC and to spend more on Netflix."
Hastings added that "there's much more growth in the consumer market than there is in advertising, which is pretty flat."
He's also expressed doubts about Netflix getting into live sports or news, which could boost the service's allure to subscribers, so that's likely out, too, at least for now.
So if Netflix is looking for other forms of near-term revenue to help support its hefty content budget ($17 billion in 2021 alone) then what can it do? There is one place that could be a revenue driver for Netflix, but if you're borrowing your mother's account you won't like it.
Netflix could crack down on password sharing - a move that the company has been considering lately.
"Basically you're going to clean up some subscribers that are free riders," Nathanson said. "That's going to help them get to a higher level of penetration, definitely, but not in long-term."
Lackluster growth is still growth
Missing projections is never good, but it's hardly the end of the world for Netflix. The company remains the market leader and most competitors are still far from taking the company on. And while Netflix's first-quarter subscriber growth wasn't great, and its forecasts for the next quarter alarmed investors, it was just one quarter.
Netflix has had subscriber misses before and it's still the most dominant name in all of streaming, and even lackluster growth is still growth. It's not as if people are canceling Netflix in droves.
Asked about Netflix's "second act" during the company's post-earnings call on Tuesday, Hastings again placed the company's focus on pleasing subscribers.
"We do want to expand. We used to do that thing shipping DVDs, and luckily we didn't get stuck with that. We didn't define that as the main thing. We define entertainment as the main thing," Hastings said.
He added that he doesn't think Netflix will have a second act in the way Amazon has had with Amazon shopping and Amazon Web Services. Rather, Netflix will continue to improve and grow on what it already does best.
"I'll bet we end with one hopefully gigantic, hopefully defensible profit pool, and continue to improve the service for our members," he said. "I wouldn't look for any large secondary pool of profits. There will be a bunch of supporting pools, like consumer products, that can be both profitable and can support the title brands.
"""

## Tokenize Text

In [144]:
tokens_input = tokenizer.encode("summarize: "+text, return_tensors='pt', 
                                max_length=tokenizer.model_max_length, 
                                truncation=True)

In [145]:
summary_ids = model.generate(tokens_input, min_length=80,
                             max_length=150,
                             length_penalty=20, 
                             num_beams=2)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [146]:
print(summary)

Netflix (NFLX) reported Tuesday it now has 208 million subscribers globally. that number missed expectations and the forecasts for its next quarter were also pretty weak. the streaming service's stock dropped as much as 8% on Wednesday, leading some to wonder what the future of the streamer looks like. if competition continues to gain strength, people start heading outdoors and if, most importantly, its growth slows, it wouldn't hurt if Netflix started thinking about other ways to make money - like selling ads.


In [147]:
# number of tokens generated from the text using T5 Tokenizer
len(tokenizer(text)['input_ids'])

Token indices sequence length is longer than the specified maximum sequence length for this model (1237 > 512). Running this sequence through the model will result in indexing errors


1237

Using BERT summarizer to extract only top 50% of sentences that are considered important.

In [148]:
bert_model = Summarizer()
ext_summary = bert_model(text, ratio=0.5)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [149]:
print(ext_summary)

New York (CNN Business)Netflix is synonymous with streaming, but its competitors have a distinct advantage that threatens the streaming leader's position at the top. Disney has Disney+, but it also has theme parks, plush Baby Yoda dolls, blockbuster Marvel movies and ESPN. It's worked out well for the company - so far. Or put another way, the company's lackluster user growth last quarter is a signal that it wouldn't hurt if Netflix - a company that's lived and died with its subscriber numbers - started thinking about other ways to make money. Not so fast
There are ways for Netflix to make money other than raising prices or adding subscribers. Netflix could have 30-second commercials on their programming or get sponsors for their biggest series and films. His reasoning: It doesn't make business sense. You know, advertising looks easy until you get in it. Then you realize you have to rip that revenue away from other places because the total ad market isn't growing, and in fact right now 

## Tokenize BERT Summary

In [150]:
tokens_input_2 = tokenizer.encode("summarize: "+ext_summary, return_tensors='pt', 
                                max_length=tokenizer.model_max_length, 
                                truncation=True)

In [151]:
len(tokenizer(ext_summary)['input_ids'])

522

## Extractive-Abstractive Summary

In [152]:
summary_ids_2 = model.generate(tokens_input_2, min_length=80,
                             max_length=150,
                             length_penalty=20, 
                             num_beams=2)

summary_2 = tokenizer.decode(summary_ids_2[0], skip_special_tokens=True)

In [153]:
print(summary_2)


Netflix's lackluster user growth last quarter is a signal that it wouldn't hurt if it started thinking about other ways to make money. the company could crack down on password sharing - a move that the company has been considering lately. "i wouldn't look for any large secondary pool of profits," says Hastings. he says he doesn't think Netflix will have a second act in the way Amazon has had with Amazon shopping and Amazon Web Services.


In [154]:
text1 = """
AMONG the numerous mechanisms that make human life possible is the body’s ability to heal wounds and regenerate damaged tissue. The process begins as soon as an injury occurs.
Consider: The healing process is made possible by a cascade of complex cellular functions:
Platelets adhere to tissues around a wound, forming a blood clot and sealing damaged blood vessels.
Inflammation protects against infection and removes any “debris” caused by the injury.
Within days, the body begins to replace injured tissue, make the wound contract, and repair damaged blood vessels.
Finally, scar tissue remodels and strengthens the damaged area.
Inspired by blood clotting, researchers are developing plastics that can “heal” 
damage to themselves. Such regenerating materials are equipped with tiny parallel 
tubes containing two chemicals that “bleed” when any damage occurs. As the two 
chemicals mix, they form a gel that spreads across the damaged areas, closing 
cracks and holes. As the gel solidifies, it forms a tough substance that restores 
the material’s original strength. One researcher admits that this synthetic healing process currently under development is “reminiscent” of what already exists in nature.
"""

In [155]:
tokens_input = tokenizer.encode("summarize: "+text1, return_tensors='pt', 
                                max_length=tokenizer.model_max_length, 
                                truncation=True)

In [156]:
summary_ids = model.generate(tokens_input, min_length=80,
                             max_length=150,
                             length_penalty=20, 
                             num_beams=2)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [157]:
print(summary)

researchers are developing plastics that can “heal” damage to themselves. inspired by blood clotting, researchers are developing plastics that can “heal” damage to themselves. regenerating materials are equipped with tiny parallel tubes containing two chemicals that “bleed” when any damage occurs. as the two chemicals mix, they form a gel that spreads across the damaged areas, closing cracks and holes. as the gel solidifies, it forms a tough substance that restores the material’s original strength.


In [158]:
# number of tokens generated from the text using T5 Tokenizer
len(tokenizer(text)['input_ids'])

1237

## Working with data frames

In [159]:
train = pd.read_csv('guardian_publications.csv')


In [161]:
#Selecting 5 rows to work on
train = train.head(5)

In [162]:
result = []
for i in range(len(train["text"])):
    result.append(tokenizer.encode("summarize: "+train["text"][i], return_tensors='pt', 
                                max_length=tokenizer.model_max_length, 
                                truncation=True))
    
    

In [163]:
train["tokens_input"]=result


In [164]:
result1 = []
for i in range(len(train["tokens_input"])):
    result1.append(model.generate(train["tokens_input"][i], min_length=80,
                             max_length=150,
                             length_penalty=20, 
                             num_beams=2))

In [165]:
train["summary_ids"]=result1


In [166]:
result2 = []
for i in range(len(train["summary_ids"])):
    result2.append(tokenizer.decode((train["summary_ids"][i])[0], skip_special_tokens=True))

In [167]:
result3 = []
for i in range(len(train["text"])):
    result3.append(len(tokenizer(train["text"][i])['input_ids']))

In [168]:
train["summary"]=result2
train["No.tokens"]=result3
train[["text","No.tokens","summary"]]

,text,No.tokens,summary
0,"Shell’s long-serving chief executive, Ben van ...",698,"van Beurden, who took over in 2014, would leav..."
1,A South African court has upheld a ban imposed...,395,"the 2014 decision granting the right for the ""..."
2,Gas shortages across Europe are likely to last...,1022,cuts to the supply of Russian gas since the in...
3,Shell has agreed to pay half a million pounds ...,680,"energy giant's consumer arm, shell energy reta..."
4,The chair of the Church of England Pensions Bo...,1405,the church of england has rejected pressure to...


## Working with Q & A

In [169]:
passage = '''Indicators for targets 9.b and 9.c have data available. Globally, 
energy efficiency and the use of cleaner fuels and technologies reduced carbon dioxide emissions 
per unit of value added by 13 per cent between 2000 and 2013. Although expenditure on research and 
development continues to grow globally, the poorest countries, especially those in Africa, spend a very small 
proportion of their GDP on such expenditure. In 2013, global investment in research and development stood at $1.7 trillion 
(purchasing power parity), up from $732 billion in 2000.
this is labeled as '''

In [170]:
result = predictor.predict(passage=passage, question="how much was the investment?")

In [171]:
result["best_span_str"]

'$1.7 trillion'

## Data Frames

In [172]:
def qestions(df,column,question):
    result = []
    for i in range(len(df[column])):
        result.append(predictor.predict(passage=train[column][i], question=question)["best_span_str"])
    return result

In [174]:
train = train[["text","No.tokens","summary"]]

In [176]:
train["answers"] = qestions(train,column= "text", question = "Is Shell involved?")
train.3+6

,text,No.tokens,summary,answers
0,"Shell’s long-serving chief executive, Ben van ...",698,"van Beurden, who took over in 2014, would leav...",Shell declined to comment on Van Beurden’s pen...
1,A South African court has upheld a ban imposed...,395,"the 2014 decision granting the right for the ""...",Shell did not say if it would appeal against t...
2,Gas shortages across Europe are likely to last...,1022,cuts to the supply of Russian gas since the in...,Shell made record profits of nearly £10bn betw...
3,Shell has agreed to pay half a million pounds ...,680,"energy giant's consumer arm, shell energy reta...",The contributions Shell has made to the redres...
4,The chair of the Church of England Pensions Bo...,1405,the church of england has rejected pressure to...,short-term expansion plans amount to about 4bn...
